## By Adrian Abeyta and Noemi Turner

In [18]:
import importlib
import numpy as np

import utils
importlib.reload(utils)
import plot_utils as pu
importlib.reload(pu)

from mypytable import MyPyTable
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [3]:
data = MyPyTable()
data.load_from_file("input_data/forbes_data.csv")

# Clean Data

In [11]:
num_rows, num_cols = data.get_shape()
y_data = utils.discritize_data_by_million(data)


# Split Data

In [12]:
X_train, X_test, y_train, y_test = myevaluation.train_test_split(data.data, y_data)

# KNN Classifier

In [19]:
knn = MyKNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)


AttributeError: module 'mysklearn.myutils' has no attribute 'compute_euclidian_distance'

# Map Data